In [41]:
import numpy as np
import pandas as pd

df = pd.read_csv("../../keras/ner_dataset.csv")

In [42]:


tags = np.unique(df.Tag)

tags

transform_tag_mapping = {
    'B-art': ['Object'],
    'B-eve': ['Event'],
    'B-geo': ['Location', 'Party'],
    'B-gpe': ['Race'],
    'B-nat': ['SpecialTerm'],
    'B-org': ['Party'],
    'B-per': ['Party'],
    'B-tim': ['Time'],
    'I-art': ['Object'],
    'I-eve': ['Event'],
    'I-geo': ['Location', 'Party'],
    'I-gpe': ['Race', "Party"],
    'I-nat': ['SpecialTerm'],
    'I-org': ['Party'],
    'I-per': ['Party'],
    'I-tim': ['TemporalUnit'],
    'O': [],
}

def transform_tag(tag):
    out = transform_tag_mapping[tag]
    out = [tag] + out
    out = "|".join(out)
    return out

df["newTag"] = df.Tag.apply(transform_tag)

df.head(30)

,Sentence #,Word,POS,Tag,newTag
0,Sentence: 1,Thousands,NNS,O,O
1,NaN,of,IN,O,O
2,NaN,demonstrators,NNS,O,O
3,NaN,have,VBP,O,O
4,NaN,marched,VBN,O,O
5,NaN,through,IN,O,O
6,NaN,London,NNP,B-geo,B-geo|Location|Party
7,NaN,to,TO,O,O
8,NaN,protest,VB,O,O
9,NaN,the,DT,O,O


In [43]:
from label_functions.basicTypes import is_Float, is_Integer
from label_functions.CountryCode import is_CountryCode
from label_functions.CryptoCurrencyCode import is_CryptoCurrencyCode
from label_functions.CurrencyCode import is_CurrencyCode
from label_functions.TemporalUnit import is_TemporalUnit
from label_functions.Timezone import is_TimeZone
from label_functions.US_States import is_US_States

In [44]:
text = "usd"

In [45]:
is_CurrencyCode(text)

True

In [46]:
class label_function:
    def __init__(self, function, tags = [], seperator = "|"):
        self.label_function = function
        self.tags = seperator.join(tags)
        self.seperator = seperator
    def label(self, text, origin_label):
        if self.label_function(text):
            return f"{origin_label}{self.seperator}{self.tags}"
        else:
            return origin_label

In [47]:
labels = [
    [is_Float, ["Float"]],
    [is_Integer, ["Integer"]],
    [is_CountryCode, ["CountryCode"]],
    [is_CryptoCurrencyCode, ["CryptoCurrencyCode"]],
    [is_CurrencyCode, ["CurrencyCode"]],
    [is_TemporalUnit, ["TemporalUnit"]],
    [is_TimeZone, ["Timezone"]],
    [is_US_States, ["US_States"]],
]

In [48]:
label_functions = []
for label in labels:
    label_functions.append(label_function(label[0], label[1]))

In [49]:
def label_with_functions(row):
    for function in label_functions:
        row["newTag"] = function.label(row.Word, row.newTag)
    return row

In [50]:
import swifter

In [51]:
%%time
df2 = df.swifter.apply(label_with_functions, axis = 1)

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

CPU times: user 13.1 s, sys: 264 ms, total: 13.4 s
Wall time: 2min 24s


In [81]:
#remove O if it have other value
df2.newTag = df2.newTag.apply(lambda x: x[2:] if "O|" == x[0:2] else x)

In [83]:
np.unique(df2.newTag)

array(['B-art|Object', 'B-art|Object|CountryCode', 'B-art|Object|Integer',
       'B-eve|Event', 'B-eve|Event|CurrencyCode', 'B-eve|Event|Integer',
       'B-geo|Location|Party', 'B-geo|Location|Party|CountryCode',
       'B-geo|Location|Party|CountryCode|Timezone',
       'B-geo|Location|Party|CurrencyCode', 'B-geo|Location|Party|Float',
       'B-geo|Location|Party|Timezone', 'B-geo|Location|Party|US_States',
       'B-gpe|Race', 'B-gpe|Race|CountryCode', 'B-gpe|Race|Timezone',
       'B-nat|SpecialTerm', 'B-nat|SpecialTerm|CurrencyCode',
       'B-nat|SpecialTerm|Integer', 'B-org|Party',
       'B-org|Party|CountryCode', 'B-org|Party|CountryCode|US_States',
       'B-org|Party|CryptoCurrencyCode', 'B-org|Party|CurrencyCode',
       'B-org|Party|Float', 'B-org|Party|Integer', 'B-org|Party|Timezone',
       'B-org|Party|US_States', 'B-per|Party',
       'B-per|Party|CountryCode|US_States', 'B-per|Party|CurrencyCode',
       'B-per|Party|Integer', 'B-per|Party|TemporalUnit',
       'B-

In [84]:
df2.to_csv("./NER_multilabel_data_v2")